In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import HashingVectorizer, CountVectorizer

from icepickle.linear_model import save_coefficients, load_coefficients
from icepickle.pipeline import make_partial_pipeline

url = "https://raw.githubusercontent.com/koaning/optimal-on-paper/main/data/outofscope-intent-classification-dataset.csv"
df = pd.read_csv(url)
X, y = list(df['text']), df['label']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y)

cv = CountVectorizer(binary=True).fit(X)

Let's see if we can use apricot to give us a subset worth labelling first. I won't use prediction metrics to discuss the quality of the subset. Instead I'll give it a dataset with 150 labels and I'll just count how many of them appear in the subset. The goal is to find a subset that "covers a lot of ground", is if there's no diversity in the retreived labels ... something is up. 

In [2]:
from apricot import FacilityLocationSelection, MaxCoverageSelection

facility_selection = FacilityLocationSelection(1000, metric='euclidean', optimizer='lazy', verbose=False)
facility_selection.fit(cv.transform(X_train).todense())

coverage_selection = MaxCoverageSelection(1000, optimizer='naive')
coverage_selection.fit(cv.transform(X_train))

/home/vincent/Development/benchmarks/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [5]:
from memo import memlist, memfile, grid, time_taken

def calc_score(y_sel):
    return {"n_unique_labels": len(set(y_sel))}

data = []

@memlist(data=data)
@time_taken()
def run_experiment(method, n):
    if method == "facility":
        selection = facility_selection.ranking[:n]
        return calc_score(y_sel=np.array(y_train)[selection])
    if method == "maxcoverage":
        selection = coverage_selection.ranking[:n]
        return calc_score(y_sel=np.array(y_train)[selection])
    if method == "random":
        return {"n_unique_labels": np.mean([len(set(np.random.randint(1, 150, (n, )))) for i in range(1000)])}

In [6]:
from tqdm import tqdm

In [7]:
for settings in tqdm(grid(method=["facility", "random"], n=list(range(1, 1000)))):
    run_experiment(**settings)

100%|████████████████████████████████████████████████████████████████████████████| 1998/1998 [01:06<00:00, 29.87it/s]


In [8]:
import altair as alt 

alt.data_transformers.disable_max_rows()


pltr = pd.DataFrame(data)

(alt.Chart(pltr)
  .mark_line()
  .encode(x='n', y='n_unique_labels', color="method")
  .properties(width=600, height=250)
  .interactive())

alt.Chart(...)

It seems that the max coverage approachs is worse than random. But the facility approach seems to be interesting. 

But let's now use the universal sentence encoder as an encoding instead of countvectors. Maybe those are "easier" to deal with.

In [9]:
from whatlies.language import UniversalSentenceLanguage

2022-05-20 20:49:38.206994: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-20 20:49:38.207016: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/vincent/Development/benchmarks/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
X_train = UniversalSentenceLanguage().fit_transform(X_train)

2022-05-20 20:49:39.750524: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-20 20:49:39.750680: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-20 20:49:39.750687: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-20 20:49:39.750702: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2022-05-20 20:49:39.750822: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

In [11]:
facility_selection = FacilityLocationSelection(1000, metric='euclidean', optimizer='lazy', verbose=False)
facility_selection.fit(X_train)

In [12]:
data = []

@memlist(data=data)
@time_taken()
def run_experiment(method, n):
    if method == "facility":
        selection = facility_selection.ranking[:n]
        return calc_score(y_sel=np.array(y_train)[selection])
    if method == "random":
        return {"n_unique_labels": np.mean([len(set(np.random.randint(1, 150, (n, )))) for i in range(1000)])}

In [13]:
for settings in tqdm(grid(method=["facility", "random"], n=list(range(1, 1000)))):
    run_experiment(**settings)

100%|████████████████████████████████████████████████████████████████████████████| 1998/1998 [01:30<00:00, 22.18it/s]


In [14]:
pltr = pd.DataFrame(data)

(alt.Chart(pltr)
  .mark_line()
  .encode(x='n', y='n_unique_labels', color="method")
  .properties(width=600, height=250)
  .interactive())

alt.Chart(...)

The universal sentence encoder indeed seems to contribute something. That's interesting. But let's now also compare against just running k-means on these embeddings.

In [15]:
from sklearn.cluster import KMeans

In [ ]:
for n in [10, 20, 50, 100, 150, 200, 300, 400, 500, 1000]:
    mod = KMeans(n)
    selection = mod.fit_transform(X_train).argmin(axis=0)
    d = {
        "n_unique_labels": len(set(np.array(y_train)[selection])),
        "method": "kmeans",
        "n": n
    }
    data.append(d)

In [ ]:
pltr = pd.DataFrame(data)

(alt.Chart(pltr)
  .mark_line()
  .encode(x='n', y='n_unique_labels', color="method")
  .properties(width=600, height=250)
  .interactive())

It seems like `Kmeans` is able to spread out as well as the facility approach. The main difference though is that the facility approach is much *faster*. 